In [176]:
import logging
import torch
import itertools, os, re, sys, time
import torch.nn as nn
import numpy as np
from functools import partial
from collections import namedtuple
from fields import Vocab, Field, Parms, Semantic

args = Parms()
args.lstm_step_num = 2
args.lstm_hid = 64
semantic = Semantic()
vocab = Vocab(semantic)
path = "./data/multi30k/vocab.txt"

vocab.load(path)
print("vocab.size:", vocab.size)

def logg_process(path):
    logger = logging.getLogger(__name__)  

    logger.setLevel(logging.INFO)

    file_handler = logging.FileHandler(path)
    formatter    = logging.Formatter('%(asctime)s : %(levelname)s : %(name)s : %(message)s')
    file_handler.setFormatter(formatter)

    logger.addHandler(file_handler)

    # Logs
    # logger.debug('A debug message')
    # logger.info('An info message')
    # logger.warning('Something is not right.')
    # logger.error('A Major error has happened.')
    # logger.critical('Fatal error. Cannot continue')
    
def log_start(path, restart = True):
    if restart:
        if os.path.isfile(path):
            os.remove(path)
        logg_process(path)
    else:
        logg_process(path)

class NLU(nn.Module):
    def __init__(self,):
        super(NLU, self).__init__()
        self.type = 'projector'


class NLU_Classify(nn.Module):
    def __init__(self):
        super(NLU_Classify, self).__init__()
        self.type = 'classifier'
        self.emb = nn.Embedding(vocab.size, embedding_dim = 128)
        self.lstm = nn.LSTM(128, args.lstm_hid, args.lstm_step_num)
#         , batch_first = True)
        self.fc = nn.Linear(64, 10)
        self.softmax = nn.Softmax(dim = 1)

    def forward(self,x):
        logger.info('inputs shape{}'.format( x.shape))
        x = self.emb(x); logger.info('emb shape {}'.format(x.shape))
        h0 = torch.randn(2,  20, args.lstm_hid)
        c0 = torch.randn(2,  20, args.lstm_hid)
        x, (hn, cn) = self.lstm(x, (h0, c0));logger.info('lstm shape {}'.format( x.shape))
        x = self.fc(x);logger.info('fc shape {}'.format( x.shape))
        x = self.softmax(x);logger.info('softmax shape{}'.format( x.shape))        
        result = torch.argmax(x,dim=1);logger.info('result:'.format(result))        
        
        return result 


log_start('./logfile.log')
ut = NLU_Classify()


n_sent, batch_size = 5, 5
sent_inds = [np.random.randint(vocab.size, size = 20) for i in range(batch_size)] 
## sent must of the same size for embed

inputs = torch.tensor(sent_inds)
output = ut(inputs)

# %cat ./logfile.log

for sent in output:
    a = [vocab.vocab_list[ind] for ind in sent]
    print(a)

vocab.size: 8699
